In [1]:
import requests
import pandas as pd


import re
import hashlib
import sqlite3
import telegram_send
import telebot
import time
import configparser

from lxml import html
from requests.exceptions import ConnectTimeout
from bs4 import BeautifulSoup
from lxml import etree
from urllib3.exceptions import MaxRetryError, SSLError

In [2]:
#Ограничивает длину слов
WORLD_LIMIT = 400

TEST_CHANEL = '-1002035663893'

In [3]:
# Создание объекта конфигурации
config = configparser.ConfigParser()

# Чтение данных из файла
config.read('config.ini')

BOT_TOKEN = config.get('tokens', 'BOT_TOKEN')

In [4]:
df_chanels = pd.DataFrame([['top_Z_museum', 1], 
                           ['high_Z_museum', 1], 
                           ['region_Z_museum', 1], 
                           ['low_Z_museum', 2],
                           ['gulag', 3],
                                                                                                               
                           ],columns=['category', 'stage'])


d_stage = {
1:'-1002045319076', 
2:'-1002014969214',
3:'-1002124668886',
}

## Функции

In [5]:
def get_div(tree, xpath_format, div_mode=True):
    list_divs = []


    # Выбор элемента
    selected_element = tree.xpath(xpath_format)


    for div in selected_element:
        # Получение HTML-разметки только для выбранного элемента
        html_representation = html.tostring(div, encoding='unicode', method='html')

        list_divs.append(html_representation)
    return list_divs


# Parse

## Большой лист

In [6]:
dict_mus = {

    
## Профильные/ тематические (мой личный топ Z музеев)
    
'Музей победы': ['https://victorymuseum.ru/playbill/',
'//div[@class="oblique-items"]//div[@class="item"]', 'top_Z_museum'],
    
'Музей заповедник Сталинградская битва': ['https://stalingrad-battle.ru/',
'//div[@class="section gray-bg fringe-top-gray fringe-bottom-gray"]//div[@class="item"]', 'top_Z_museum'],
    
'Музей блокады Ленинграда': ['https://blokadamus.ru/музей-обороны-и-блокады-ленинграда/события/',
'//*[@class="elementor-post__card"]', 'top_Z_museum'],
    
'Музей минобороны в Москве': ['http://www.cmaf.ru/',
'//*[@class="reg-news"]//p[@class="news-item"]/following-sibling::*[1]', 'top_Z_museum'],
    
'Музей дорогами победы (парк Патриот)': ['https://1418museum.ru/news/',
'//*[@class="newsItem__contains"]', 'top_Z_museum'],
    
'Бородинская битва': ['https://1812panorama.ru/afisha',
'//*[@class="field-more-title"]', 'top_Z_museum'],
    
    
    
    
    
## Столичные чуть менее Z
    
'Музей современной истории России': ['https://sovrhistory.ru/exhibitions/future/index.php',
'//*[@class="exhibitions__text"]', 'high_Z_museum'],
    
'Выставочный центр Манеж, Москва': ['https://moscowmanege.ru/ru/vystavki-all/',
'//*[@class="news-block exhibition"]', 'high_Z_museum'],
    
'Музей Москвы': ['https://mosmuseum.ru/exhibitions/',
'//*[@class="articles-grid"]//*[@class="content"]', 'high_Z_museum'],
    
'ГИМ, Москва': ['https://mediashm.ru/?all=1', '//a[@class="element"]', 'high_Z_museum'],
    
'Музей кино, Москва': ['https://www.museikino.ru/media_center/',
'//div[@class="media__inf"]', 'high_Z_museum'],
    
    
## Региональные Z  

'Тульский музей оружия ': ['https://www.museum-arms.ru/news/?str1=2024',
'//div[@class="newsfirst"]', 'region_Z_museum'],
    
'Музей военной истории «Свердловск: Говорит Москва!» ': ['https://ugvim.ru/news/',
'//a[@class="front_cont h_more_w"]', 'region_Z_museum'],
    
'Дом-музей В.И. Ленина': ['https://www.leninmemorial.ru/news/',
'//div[@class="row row-news-page"]', 'region_Z_museum'],
    
'Мемориал «В честь героев Курской битвы»': ['http://мемориал.бгикм.рф/меню/новости',
'//div[@class="post"]', 'region_Z_museum'],
    
'Мемориальный музей военного и трудового подвига 1941–1945 гг': ['http://patriotrm.clan.su/news/?page1',
'//td[@style="padding:3px;"]', 'region_Z_museum'],
    
'ГОСУДАРСТВЕННЫЙ ВОЕННО-ИСТОРИЧЕСКИЙ МУЗЕЙ-ЗАПОВЕДНИК «ПРОХОРОВСКОЕ ПОЛЕ»': ['https://прохоровское-поле.рф/vystavochnaya-deyatelnost.html',
'//div[@class="news-item"]', 'region_Z_museum'],
    
'Государственный музей политической истории России ': [f'https://polithistory.ru/news-{pd.to_datetime("now").year}',
'//div[@class="sc_news_items "]/article', 'region_Z_museum'],
    
    
'Музейно-мемориальный комплекс «История танка Т-34»': ['https://museum-t-34.ru/новости/',
'//div[@class="site_content_left"]', 'region_Z_museum'],
    
'Музей «Боевая слава» г. Дербента': ['https://derbentmuseum.ru/',
'//section[@class="blog"]//div[@class="col-lg-4"]', 'region_Z_museum'],
    
'Музей «Мемориал Победы»': ['https://memorial24.ru/novosti',
'//div[@class="novelty clearfix novelty-teaser"]', 'region_Z_museum'],
    
'Музей «Смоленщина в годы Великой Отечественной войны 1941−1945 гг.': ['https://www.smolensk-museum.ru/novosti/muzeynaya_zhizn/',
'//div[@class="news__item"]', 'region_Z_museum'],
    
'Музей «Невский пятачок': ['https://www.lenoblmus.ru/news',
'//li[@class="col-md-4 col-lg-3 col-sm-6 col-xs-12"]', 'region_Z_museum'],
    
'Музей «Дорога жизни»': ['https://navalmuseum.ru/news',
'//div[@class="content-box"]', 'region_Z_museum'],


## Региональные исторические/краеведческие музеи, в которых что то проскакивает
    
'Самарский областной историко-краеведческий музей им. П.В. Алабина': ['https://alabin.ru/',
'//div[@class="news__post"]', 'low_Z_museum'],
    
'Якутский государственный объединенный музей истории и культуры народов Севера им. Ем. Ярославского': ['https://yakutmuseum.ru/',
'//a[@class="news-item"]', 'low_Z_museum'],
    
'Краснодарский государственный историко-археологический музей-заповедник им. Е.Д. Фелицына': ['https://felicina.ru/news/',
'//div[@class="content-area content-left col-md-8 col-sm-8"]//article', 'low_Z_museum'],
    
'Государственный исторический музей Южного Урала': ['http://www.chelmuseum.ru/news/#content',
'//div[@class="card"]', 'low_Z_museum'],
    
'Тамбовский областной краеведческий музей': ['https://tmbmuseum.ru/mainmenu/news/relevant',
'//h3', 'low_Z_museum'],
    
'Национальный музей Республики Башкортостан ': ['https://museumrb.ru/category/afisha/events/',
'//div[@class="upcoming-event-2 pt-80 pb-80"]//div[@class="single-event single-event-3 mb-30 wow fadeInUp my-single-event-3"]', 'low_Z_museum'],
    
'Ставропольский государственный музей-заповедник': ['https://stavmuseum.ru/',
'//div[@class="anonsiblock"]', 'low_Z_museum'],
    
'Новосибирский государственный краеведческий музей': ['https://youmuseum.ru/news/',
'//div[@class="text"]', 'low_Z_museum'],

'Белгородский государственный историко-краеведческий музей ': ['http://бгикм.рф/news/',
'//div[@class="news_elem_container"]', 'low_Z_museum'],
    
'Оренбургский губернаторский историко-краеведческий музей ': ['http://ogikm.ru/',
'//div[@class="tab-content"]//div[@class="item"]', 'low_Z_museum'],
    
'Омский государственный историко-краеведческий музей': ['https://sibmuseum.ru/news/',
'//a[@class="news-card__link"]', 'low_Z_museum'],
    
'Национальный музей Удмуртской Республики им. К. Герда': ['https://nmur.ru/afisha',
'//div[@class="single-block"]', 'low_Z_museum'] ,
    
'Саратовский областной музей краеведения': ['http://www.comk.ru/museum/news/',
'//div[@class="news-item"]', 'low_Z_museum'] ,   

'Национальный музей Республики Марий Эл им. Т. Евсеева ': ['https://www.fumus.ru/events/',
'//ul[@class="events__list"]/li', 'low_Z_museum'] ,  
    
'Ростовский областной музей краеведения ': ['https://www.rostovmuseum.ru/calendar/list-of-events/',
'//div[@class="row"]//div[@class="s-news-wrapper"]', 'low_Z_museum'] ,  

'Воронежский областной краеведческий музей ': ['https://museum-vrn.ru/muzej/novosti/',
'//a[@class="i-wrap"]', 'low_Z_museum'] , 
    
'Национальный музей Республики Татарстан ': ['https://tatmuseum.ru/news/?PAGEN_1=4',
'//a[@class="news-item"]', 'low_Z_museum'] , 

'Брянский государственный краеведческий музей ': ['https://bgkm.ru/',
'//div[@class="bloglatestnews1"]/div', 'low_Z_museum'] , 
    
'Музей истории Бурятии им. М.Н. Хангалова ': ['https://muzeyrb.ru/sobytiya-1?page=2',
'//article[@class="col-lg-4 mb-50 animate-conner"]', 'low_Z_museum'] , 
    
'Национальный музей Республики Дагестан им. А. Тахо-Годи ': ['https://dagmuzey.ru/events',
'//a[@class="s-events-wrapper__item s-events-wrapper-item"]', 'low_Z_museum'] , 
    
'Ульяновский областной краеведческий музей имени И.А. Гончарова ': ['https://uokm.site/category/уокм/новости/',
'//a[@class="a news-link"]', 'low_Z_museum'], 
    
'Музей истории города Улан-Удэ ': ['https://uumuseum.ru/news/',
'//a[@class="event-index-item"]', 'low_Z_museum'], 

'Национальный музей Республики Коми ': ['https://museumkomi.ru/?cat=3',
'//div[@class="special-category"]', 'low_Z_museum'], 

'Национальный музей Республики Алтай имени А.В. Анохина ': ['http://musey-anohina.ru/index.php/novosti',
'//div[@class="catItemBody"]', 'low_Z_museum'], 
    
'Чувашский национальный музей ': ['https://chnmuseum.ru/news/',
'//main[@class="site-main"]/article', 'low_Z_museum'],   
    
'Мордовский республиканский объединенный краеведческий музей им. И.Д. Воронина ': ['http://mrkm.ru/novosti/?PAGEN_1=4',
'//a[@class="box news"]', 'low_Z_museum'],  
    
'Национальный музей им. Алдан Маадыр Республики Тыва ': ['https://музейтыва.рф/?cat=16&paged=1',
'//div[@class="main col-lg-9 col-md-8  postlist"]/article', 'low_Z_museum'], 

'Хабаровский краевой музей имени Н.И. Гродекова ': ['https://hkm.ru/news/',
'//a[@class="new-item list-item"]', 'low_Z_museum'], 
    

'Калининградский областной историко-художественный музей ': ['https://koihm.ru/category/novosti/',
'//div[@class="article-inner"]', 'low_Z_museum'], 
    
'Национальный музей Республики Карелия ': ['https://kareliamuseum.ru/news/',
'//article[@class="clearfix l-news-article"]', 'low_Z_museum'], 
    
'Астраханский музей заповедник ': ['https://astmuseum.ru/ru/news/',
'//div[@class="iblock-list-item-wrap  "]', 'low_Z_museum'], 
    
'Сургутский краеведческий музей ': ['https://skmuseum.ru/',
'//ul[@class="is-layout-flow wp-block-post-template has-small-font-size"]/li', 'low_Z_museum'], 

'Магаданский областной краеведческий музей ': ['https://magadanmuseum.ru/index.php?do=cat&category=news',
'//div[@id="dle-content"]/table', 'low_Z_museum'], 

'Курский областной краеведческий музей ': ['http://kursk-museum.ru/',
'//div[@class="clearfix news_content"]', 'low_Z_museum'],  
    
'Национальный музей Республики Адыгея ': ['https://adyg-museum.ru/ru/',
'//div[@class=" col-12 col-sm-12 col-md-6 col-lg-4 col-xl-4"]|//div[@class=" col-12 col-sm-12 col-md-12 col-lg-6 col-xl-6"]', 'low_Z_museum'],  

'Музейно-выставочный центр г. Иваново ': ['http://www.igikm.ru/sobytiya/novosti/',
'//p[@class="announce-text"]', 'low_Z_museum'], 
    
## ГУЛАГ
    
  
'Музей Анны Ахматовой в Фонтанном доме': ['https://www.akhmatova.spb.ru/afisha',
'//a[@class="js-product-link"]', 'gulag'],
    
'Музей Гулага': ['https://gmig.ru/museum/news/',
'//a[@class="news-in-list"]', 'gulag'],
    
'Соловецкий государственный историко-архитектурный и природный музей-заповедник': ['http://solovky.ru/ru/vystavki/v-rossii-peredvizhnye-vystavki',
'//div[@class="field field-name-field-expandable-block field-type-field-collection field-label-hidden"]/div[@class="field-items"]/div', 'gulag'],

'Новый музей обороны и блокады Ленинграда, АНАСТАСИЯ КАРКАЧЕВА О ВЫСТАВКЕ В НОВОМ МУЗЕЕ': ['https://www.colta.ru/articles/art/14086-novyy-muzey-oborony-i-blokady-leningrada',
'//div[@class="mask left_posts"]/article', 'gulag'],
    
'Пермь 36': ['https://mperm36.ru/novosti',
'//div[@class="news-row"]/div', 'gulag'],
    
'Военно-медицинский музей': ['https://milmed.spb.ru/category/новости/',
'//div[@class="col_block"]', 'gulag'],
    
    

    
    
    
        
    
    
    
  
                                                   }

In [7]:
dict_info = {}


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

for name in dict_mus:
    site = dict_mus[name][0]
    xpath = dict_mus[name][1]
    group_name = dict_mus[name][2]
    
    try:
        response = requests.get(site,  verify=False
                                , headers=headers
                               )
    except ConnectTimeout:
        print(f"Время подключения к {site} истекло.")
        continue
        
    except ConnectTimeout:
        print(f"Произошла ошибка MaxRetryError: {site}")
        continue
        
    except requests.exceptions.SSLError as e:
        print(f"Произошла ошибка SSLError: {site}")
        continue

    if response.status_code == 200:
        # Создаем объект дерева HTML из содержимого ответа
        tree = html.fromstring(response.text)
        
        
        target_obj = get_div(tree, xpath)  
        dict_info[site] = target_obj
        
    elif response.status_code == 404:    
        # Создаем объект дерева HTML из содержимого ответа
        print(f'Ошибка 404 для сайта  {site}')
        
        
    else:
        print(f'Ошибка статуса {response.status_code} для {site}')

C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'victorymuseum.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'stalingrad-battle.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'blokadamus.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\con

Ошибка 404 для сайта  https://polithistory.ru/news-2024


C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'museum-t-34.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'derbentmuseum.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'memorial24.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectio

C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hkm.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'koihm.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kareliamuseum.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056

In [8]:
df_raw_p = pd.DataFrame([dict_info]).T

In [9]:
df_raw_p = df_raw_p.explode(0)
df_raw_p = df_raw_p.reset_index()

In [10]:
df_raw_p = df_raw_p.rename(columns={'index':'site', 0:'raw_str'})

In [11]:
df_raw_p[df_raw_p['raw_str'].isna()]

,site,raw_str
1144,https://www.colta.ru/articles/art/14086-novyy-...,NaN


In [12]:
df_raw_p

,site,raw_str
0,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_263216"" da..."
1,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_263213"" da..."
2,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_244446"" da..."
3,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_263209"" da..."
4,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_181829"" da..."
...,...,...
1153,https://mperm36.ru/novosti,"<div data-id="""" class=""exposition-item"">\n ..."
1154,https://milmed.spb.ru/category/новости/,"<div class=""col_block"" style=""cursor:pointer;""..."
1155,https://milmed.spb.ru/category/новости/,"<div class=""col_block"" style=""cursor:pointer;""..."
1156,https://milmed.spb.ru/category/новости/,"<div class=""col_block"" style=""cursor:pointer;""..."


In [13]:
df_raw_p = df_raw_p[df_raw_p['raw_str'].notna()].copy()

In [14]:
df_raw_p['text_elem'] = df_raw_p['raw_str'].apply(lambda x: BeautifulSoup(x, 'html.parser').text)

## Доработка запросов

In [15]:
target_site = 'https://www.akhmatova.spb.ru/afisha'
temp_s = set()

# Проверяю есть ли вообще такие
for i in dict_mus.values():
    temp_s.add(i[0])
#assert target_site in temp_s




if any(df_raw_p.site == target_site):
    # Оставляю 10 первых значений 
    part_target = df_raw_p[df_raw_p['site']==target_site][:10]
    
    part_all = df_raw_p[df_raw_p['site']!=target_site]
    
    df_raw_p = pd.concat([part_target, part_all]).copy()

In [16]:
df_raw_p = df_raw_p.reset_index(drop=True).copy()

## Делаем колоку с хэшом

In [17]:
def generate_hash(input_string):
    # Создаем объект хэша SHA-256
    hash_object = hashlib.sha256()

    # Преобразуем входную строку в байты и обновляем хэш
    hash_object.update(input_string.encode('utf-8'))

    # Получаем шестнадцатеричное представление хэша
    hash_value = hash_object.hexdigest()

    return hash_value

In [18]:
# Колонка для хеширования
df_raw_p.insert(0,'hash_col', df_raw_p['site']+ ' ' + df_raw_p['text_elem'])

df_raw_p['hash_col'] = df_raw_p['hash_col'].apply(generate_hash)

In [19]:
df_raw_p['text_elem'] = df_raw_p['text_elem'].str.replace('❾', '')

In [20]:
df_raw_p = df_raw_p.drop_duplicates(['site', 'text_elem']).copy()

In [21]:
df_raw_p.insert(0,'date_stamp', pd.to_datetime('today'))

## Загружаем в db

In [22]:
conn = sqlite3.connect('museums.db')

In [23]:
# Создаю таблицы

qu = '''
    CREATE TABLE if not EXISTS "parse" (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE,
    "site" TEXT,
    "raw_str" TEXT,
    "text_elem" TEXT
)
'''
conn.execute(qu)

qu = '''
    CREATE TABLE if not EXISTS "parse_temp" (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE,
    "site" TEXT,
    "raw_str" TEXT,
    "text_elem" TEXT

)
'''
conn.execute(qu)

# Добавляю таблицу, для успешных выполнений
qu = '''
    CREATE TABLE if not EXISTS send_telegram (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE, 
    "chanel_id" TEXT, 
    teleg_mes TEXT
)
'''
conn.execute(qu)

In [24]:
# Временная таблица
df_raw_p.to_sql('parse_temp', conn, index=False, if_exists='replace')


# Импортирую значения с новыми значениями по хэшу
qu = '''
INSERT INTO parse
select *
from parse_temp
where hash_col not in (select hash_col from parse)

'''
conn.execute(qu)

In [25]:
# Не загруженые значения
qu = '''
select *
from parse
where hash_col not in (select hash_col from send_telegram)
order by date_stamp
'''
df_n_compl = pd.read_sql(qu, conn)

# Чистка

In [26]:
def adj_lim(x:list):
    '''Ограничивает длину слова'''
    for i, el  in enumerate(x):

        if len(el) > WORLD_LIMIT:
            x[i] = el[:WORLD_LIMIT]+'...'
            
    return x



list_unuse = ['\t\t\t\tЧитать далее»\t\t\t', 'подробнее >>', 'Подробнее', 'Заказать', 'подробнее...']


def drop_words(x:list):
    '''
    Удаляю лишние слова 
    '''
    new_list = []
    for l in x:

        if not l in list_unuse and not bool(re.match(r'^[0-9 /]+$', l)) and l:
            new_list.append(l)
#             print('********************')
#             print(l)
#             print('********************')
#         else:
#             print('********************')
#             print('Не лобавлено',l)
#             print('********************')
    return new_list

        

In [27]:
# Удаляю https://vk.com иза ошибки
# df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace(r'Ссылка: https://vk.com[^\\]*', '', regex=True)

In [28]:
# Убираю \r
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('\r', '')

In [29]:
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('[0-9]* Comments', '', regex=True)

df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('Читать далее\W*', '', regex=True)

df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('Подробнее\W*', '', regex=True)

# Сплит строки
df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(lambda x: re.split(r"(\n)", x))

# Ограничивает длину
df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(adj_lim)

# Удаление лишних слов 
df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(drop_words)

df_n_compl['text_elem']= df_n_compl['text_elem'].apply(lambda x: ''.join(x))

In [30]:
# Удаление \n в конце текста
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('[(\\n)(\\t)]+$', '', regex=True)

# Удаление \n в начале
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('^[(\\n)(\\t)]+', '', regex=True)

# Замена \n\n на меньшее число
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('(\\n){3,}','\n\n', regex=True)

df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace(' {3,}',' ', regex=True)

# Telegram

## Добавляем имена музеев

In [31]:
d_mus_names = {}

for k in dict_mus:
    d_mus_names[dict_mus[k][0]] = k

In [32]:
df_n_compl['mus_name'] = df_n_compl['site'].map(d_mus_names)

df_n_compl['mus_name']= df_n_compl['mus_name'].fillna(df_n_compl['site'])


## Отправка

### Форматирование

### Ссылка на музей

In [33]:
df_n_compl['teleg_mes'] = '[' + df_n_compl.mus_name + ']' + '(' + df_n_compl.site + ')' + '\n'

df_n_compl['teleg_mes'] = df_n_compl['teleg_mes']+ df_n_compl.text_elem

In [34]:
df_n_compl = df_n_compl[df_n_compl['text_elem']!='']

In [35]:
df_n_compl

,date_stamp,hash_col,site,raw_str,text_elem,mus_name,teleg_mes
5,2024-01-12 23:47:13.232288,4c5f8723f68d80c50d174047412eb8a576227f4b344d81...,https://www.akhmatova.spb.ru/afisha,"<a class=""js-product-link"" href=""/pod-krovley""...",14+Регулярно«Под кровлей Фонтанного Дома...» ...,Музей Анны Ахматовой в Фонтанном доме,[Музей Анны Ахматовой в Фонтанном доме](https:...
6,2024-01-12 23:47:13.232288,0afca336066d1bd85f671fb588c6acd537c5b7296d1769...,https://www.akhmatova.spb.ru/afisha,"<a class=""js-product-link"" href=""/chast-rechi""...",18+Регулярно«Часть речи» Пушкинская картаЭкск...,Музей Анны Ахматовой в Фонтанном доме,[Музей Анны Ахматовой в Фонтанном доме](https:...
7,2024-01-12 23:47:13.232288,89d45dbc476577d6be901c687ac89a004673fd63d268f7...,https://www.akhmatova.spb.ru/afisha,"<a class=""js-product-link"" href=""/ludi-vse-byv...",6+23 декабря 2023 – 28 января 2024Люди все бы...,Музей Анны Ахматовой в Фонтанном доме,[Музей Анны Ахматовой в Фонтанном доме](https:...
8,2024-01-12 23:47:13.232288,9aed51f0ceb846c37806d7fdc6329e26852b878bc4448d...,https://www.akhmatova.spb.ru/afisha,"<a class=""js-product-link"" href=""/gomeostaz""> ...",14+27 декабря 2023 – 28 января 2024Гомеостаз....,Музей Анны Ахматовой в Фонтанном доме,[Музей Анны Ахматовой в Фонтанном доме](https:...
9,2024-01-12 23:47:13.232288,43f8e30cc86971b747637fc1f25b6ed555dd4015bc9f2d...,https://www.akhmatova.spb.ru/afisha,"<a class=""js-product-link"" href=""/ya-geniy""> <...","14+11 и 25 января 2024, 17:00«Я гений, но не ...",Музей Анны Ахматовой в Фонтанном доме,[Музей Анны Ахматовой в Фонтанном доме](https:...
...,...,...,...,...,...,...,...
63,2024-01-12 23:47:13.232288,ba5af6c072d562988cf26d783d161abeb2e480682c740d...,http://kursk-museum.ru/,"<div class=""clearfix news_content"">\r\n\t\t\t\...","Ночь, улица, фонарь, аптека…\n\nВ 1912 году Ал...",Курский областной краеведческий музей,[Курский областной краеведческий музей ](http:...
64,2024-01-12 23:47:13.232288,a6a52287cf552945424a735e3d31ac2582b63ccbbf39a1...,http://kursk-museum.ru/,"<div class=""clearfix news_content"">\r\n\t\t\t\...",В Музее кино показывали киноленту «1941. Оборо...,Курский областной краеведческий музей,[Курский областной краеведческий музей ](http:...
65,2024-01-12 23:47:13.232288,d3247dc0d6c8741cfec1a1bc54f5627b18916d6dd16c8f...,https://gmig.ru/museum/news/,"<a class=""news-in-list"" href=""/museum/news/noc...",НОЧЬ ИСКУССТВ 2023\n31 октября 2023,Музей Гулага,[Музей Гулага](https://gmig.ru/museum/news/)\n...
66,2024-01-12 23:47:13.232288,935c335ba06780148359373527ac71aa60bb66be7abe3c...,https://mperm36.ru/novosti,"<div data-id="""" class=""exposition-item"">\n ...",Приглашаем всех желающих посетить музей с экск...,Пермь 36,[Пермь 36](https://mperm36.ru/novosti)\nПригла...


### Добавляем группы

In [36]:
df_chanels

,category,stage
0,top_Z_museum,1
1,high_Z_museum,1
2,region_Z_museum,1
3,low_Z_museum,2
4,gulag,3


In [37]:
df_chanels['chanel_id'] = df_chanels['stage'].map(d_stage)

In [38]:
df_chanels

,category,stage,chanel_id
0,top_Z_museum,1,-1002045319076
1,high_Z_museum,1,-1002045319076
2,region_Z_museum,1,-1002045319076
3,low_Z_museum,2,-1002014969214
4,gulag,3,-1002124668886


In [39]:
df_mus_info = pd.DataFrame(dict_mus).T
df_mus_info.columns = ['site', 'xpath', 'category']

In [40]:
df_mus_info = df_mus_info.merge(df_chanels, right_on=['category'], left_on=['category'], how='left')

In [41]:
df_mus_info = df_mus_info[['site', 'chanel_id']]

In [42]:
df_n_compl = df_n_compl.merge(df_mus_info, on='site', how='left')

In [43]:
# assert df_n_compl[df_n_compl['chanel_id'].isna()].empty


# # Времяно!!!
df_n_compl = df_n_compl[~df_n_compl['chanel_id'].isna()]

In [44]:
df_n_compl['chanel_id'].value_counts()

chanel_id
-1002014969214    40
-1002045319076    14
-1002124668886     9
Name: count, dtype: int64

In [45]:
df_mes = df_n_compl[['hash_col', 'teleg_mes', 'chanel_id']].copy()

In [46]:
df_mes

,hash_col,teleg_mes,chanel_id
0,4c5f8723f68d80c50d174047412eb8a576227f4b344d81...,[Музей Анны Ахматовой в Фонтанном доме](https:...,-1002124668886
1,0afca336066d1bd85f671fb588c6acd537c5b7296d1769...,[Музей Анны Ахматовой в Фонтанном доме](https:...,-1002124668886
2,89d45dbc476577d6be901c687ac89a004673fd63d268f7...,[Музей Анны Ахматовой в Фонтанном доме](https:...,-1002124668886
3,9aed51f0ceb846c37806d7fdc6329e26852b878bc4448d...,[Музей Анны Ахматовой в Фонтанном доме](https:...,-1002124668886
4,43f8e30cc86971b747637fc1f25b6ed555dd4015bc9f2d...,[Музей Анны Ахматовой в Фонтанном доме](https:...,-1002124668886
...,...,...,...
58,ba5af6c072d562988cf26d783d161abeb2e480682c740d...,[Курский областной краеведческий музей ](http:...,-1002014969214
59,a6a52287cf552945424a735e3d31ac2582b63ccbbf39a1...,[Курский областной краеведческий музей ](http:...,-1002014969214
60,d3247dc0d6c8741cfec1a1bc54f5627b18916d6dd16c8f...,[Музей Гулага](https://gmig.ru/museum/news/)\n...,-1002124668886
61,935c335ba06780148359373527ac71aa60bb66be7abe3c...,[Пермь 36](https://mperm36.ru/novosti)\nПригла...,-1002124668886


In [47]:
df_mes.shape

(63, 3)

### Исполнение

In [48]:
df_mes['teleg_mes'] = df_mes['teleg_mes'].str.replace('_-', '-', regex=True)

df_mes['teleg_mes'] = df_mes['teleg_mes'].str.replace('\*', '', regex=True)

# Убирайю символ не разрывного пробела
df_mes['teleg_mes'] = df_mes['teleg_mes'].str.replace('\xa0', ' ', regex=True)

In [49]:
execute_session = pd.to_datetime('now')


# Замените 'YOUR_BOT_TOKEN' на токен вашего бота
bot = telebot.TeleBot(BOT_TOKEN)



PAUSE_ITER = 20
PAUSE_TIME = 50


# Ожидание перд запуском
# time.sleep(PAUSE_TIME)
for num, tuple_mes in enumerate(df_mes.itertuples()):
    
    try:
        # Отправка сообщения
        bot.send_message(tuple_mes.chanel_id, tuple_mes.teleg_mes, parse_mode='Markdown')
    
        # Сохранение успешных сообщений
        df = pd.DataFrame([tuple_mes]).drop(columns='Index')

        df['date_stamp'] = execute_session

        if num % 19==0 and num!=0:
            print(f'Выполнено {num} из {df_mes.shape[0]}')
            time.sleep(PAUSE_TIME)

        df.to_sql('send_telegram', conn, if_exists='append', index=False)
    except Exception as send_error:
        print(num, send_error, tuple_mes)
        df_error = pd.DataFrame([[tuple_mes.hash_col, num, send_error, 
                    tuple_mes.teleg_mes, tuple_mes.chanel_id]], columns=['hash_col','num', 'error', 'teleg_mes', 'chanel_id'])

        df_error.insert(0,'date_time', execute_session)
        df_error['error'] = df_error['error'].astype('str')
        df_error.to_sql('send_error', conn, index=False, if_exists='append')

    
    


Выполнено 19 из 63
Выполнено 38 из 63
Выполнено 57 из 63


In [50]:
num

62

In [51]:
num // 19

3

In [52]:
num % 19

5

# Тест

In [ ]:
cnjg

In [53]:
import asyncio
import async_timeout
import aiohttp
import nest_asyncio
nest_asyncio.apply()
import json
from pyppeteer import launch

In [54]:
import asyncio
from pyppeteer import launch


async def main():
   browserObj =await launch({"headless": False})
   url = await browserObj.newPage()
   await url.goto('https://scrapeme.live/shop/')


   ## Get HTML
   htmlContent = await url.content()
   await browserObj.close()
   return htmlContent


response = asyncio.get_event_loop().run_until_complete(main())
print(response)


<!DOCTYPE html><html lang="en-GB"><head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=2.0">
<link rel="profile" href="http://gmpg.org/xfn/11">
<link rel="pingback" href="https://scrapeme.live/xmlrpc.php">

<title>Products – ScrapeMe</title>
<link rel="dns-prefetch" href="//fonts.googleapis.com">
<link rel="dns-prefetch" href="//s.w.org">
<link rel="alternate" type="application/rss+xml" title="ScrapeMe » Feed" href="https://scrapeme.live/feed/">
<link rel="alternate" type="application/rss+xml" title="ScrapeMe » Comments Feed" href="https://scrapeme.live/comments/feed/">
<link rel="alternate" type="application/rss+xml" title="ScrapeMe » Products Feed" href="https://scrapeme.live/shop/feed/">
		<script type="text/javascript">
			window._wpemojiSettings = {"baseUrl":"https:\/\/s.w.org\/images\/core\/emoji\/11\/72x72\/","ext":".png","svgUrl":"https:\/\/s.w.org\/images\/core\/emoji\/11\/svg\/","svgExt":".svg","source":{"concatemoji"

In [55]:
asyncio.get_event_loop().run_until_complete(main())

NetworkError: Protocol error Page.navigate: Target closed.

In [ ]:
cnjg

In [ ]:
q = df_error['teleg_mes'][0]

In [ ]:
q

In [ ]:
q = '[Музей «Мемориал Победы»](https://memorial24.ru/novosti)\nПриглашаем в Единый Почетный караул\n\n14.10.2023\n\n4-го ноября постовцы советских лет заступят в караул вместе с участниками возрожденного Поста №1. Участие бесплатное, но нужна предварительная регистрация (ссылка ниже )\nМузей «Мемориал Победы» и Пост №1 приглашают тех, кто нес караульную службу в 1968-1992 годы, принять участие в Едином Почетном карауле.\nДвижение юнармейских Постов ведет свою историю с осени 1965-го года, когда в Волгограде был создан первый в стране пионерско-комсомольский Пост №1.\n5-го октября 1968-го года Пост №1 был открыт в Красноярске. До 1992-го года лучшие пионеры и комсомольцы города несли службу у Вечного огня на Красной площади Красноярска. Возрождение Поста №1 и караульной службы у Вечного огня на площади Мемориала Победы стало с февраля 2015-го года.\nЗаявки на участие в едином почётном карауле принимаются до 27-го октября по телефону 201-89-29 или регистрируйтесь здесь: https://vk.com/app6013442_-26051136?form_id=3#form_id=3\n\nПОДЕЛИТЬСЯ:'

In [ ]:
bot.send_message(TEST_CHANEL, q, parse_mode='Markdown')